In [ ]:
'''Imports'''
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from prettytable import PrettyTable

'''Import Utils'''


'''Import Data'''
from sklearn.datasets import load_digits, fetch_california_housing

'''Import Data Processing Utilities'''
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, f1_score
from sklearn.decomposition import PCA

'''Import Predictors'''
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from skopt import BayesSearchCV

'''Import Tuning Utilities'''
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.plots import plot_objective, plot_histogram, plot_convergence, plot_gaussian_process
from sklearn.model_selection import GridSearchCV


In [ ]:
'''Global Variables'''

scoringCriteria = [
    'positive_likelihood_ratio', 'f1_micro', 'v_measure_score', 'roc_auc_ovr_weighted',
    'adjusted_rand_score', 'neg_mean_squared_log_error', 'r2', 'precision_macro',
    'roc_auc_ovo_weighted', 'neg_median_absolute_error', 'matthews_corrcoef', 'precision_samples',
    'jaccard_micro', 'balanced_accuracy', 'precision_micro', 'recall_micro', 'f1_samples',
    'recall', 'jaccard', 'neg_mean_poisson_deviance', 'rand_score', 'jaccard_weighted',
    'neg_mean_absolute_percentage_error', 'precision_weighted', 'average_precision',
    'neg_log_loss', 'recall_samples', 'recall_weighted', 'accuracy', 'homogeneity_score',
    'neg_mean_absolute_error', 'adjusted_mutual_info_score', 'roc_auc', 'completeness_score',
    'f1_macro', 'roc_auc_ovo', 'recall_macro', 'top_k_accuracy', 'f1', 'roc_auc_ovr',
    'normalized_mutual_info_score', 'fowlkes_mallows_score', 'mutual_info_score', 'explained_variance',
    'max_error', 'neg_negative_likelihood_ratio', 'neg_brier_score', 'neg_mean_squared_error',
    'f1_weighted', 'neg_root_mean_squared_error', 'jaccard_samples', 'neg_mean_gamma_deviance',
    'jaccard_macro', 'precision'
    ]

In [ ]:
'''Classifier data EDA'''
digits = load_digits()
# print(digits.keys())
# print(digits['data'][0])
# print(digits.target)

In [ ]:
'''Default Classifier Performance'''
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=42)
for model in [DecisionTreeClassifier, RandomForestClassifier]:
    pipe = Pipeline([
        ('scale', StandardScaler()),
        # ('pca', PCA()),
        ('classify',model())],
    verbose=True)
    predicted = pipe.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, predicted))
    sns.heatmap(confusion_matrix(y_test, predicted))
    plt.title('Confusion Matrix')
    plt.show()



In [ ]:
'''Tuned Classifier Performance'''

# TODO: Model Tuning 
# Update the below to implement Bayesian Optimization model tuning.
# Use skopt.gp_minimize() vis a vi: https://machinelearningmastery.com/scikit-optimize-for-hyperparameter-tuning-in-machine-learning/
def f1(estimator, X_test, y_test):
    return f1_score(y_test, estimator.predict(X_test))

classifiers = {
    DecisionTreeClassifier.__name__: {
        'model': DecisionTreeClassifier,
        'paramSpace': {
            'classify__criterion': Categorical(['gini', 'entropy', 'log_loss']),
            'classify__splitter': Categorical(['best', 'random']),
            'classify__max_depth': Integer(1, 1000),
            'classify__min_samples_split': Real(0.01, 0.9),
            'classify__min_samples_leaf': Real(0.01, 0.9),
            'classify__min_weight_fraction_leaf': Real(0.0,0.5),
            'classify__max_features': Real(0.01,0.9),
            'classify__max_leaf_nodes': Integer(2, 4000), 
            'classify__min_impurity_decrease': Real(0.0, 1.0),
            'classify__ccp_alpha': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(digits.data[0])),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    },
    RandomForestClassifier.__name__: {
        'model': RandomForestClassifier,
        'paramSpace': {
            'classify__n_estimators': Integer(10, 2000),
            'classify__criterion': Categorical(['gini', 'entropy', 'log_loss']),
            'classify__max_depth': Integer(1, 1000), 
            'classify__min_samples_split': Real(0.01, 0.9), 
            'classify__min_samples_leaf': Real(0.01, 0.9),
            'classify__min_weight_fraction_leaf': Real(0.01,0.5),
            'classify__max_features': Real(0.01,0.9),
            'classify__max_leaf_nodes': Integer(1,2000),
            'classify__min_impurity_decrease': Real(0.01,0.9),
            # 'classify__bootstrap': Categorical([True, False]),
            'classify__oob_score': Categorical([True, False]),
            'classify__warm_start': Categorical([True, False]),
            'classify__max_samples':Real(0.01,0.9),
            'pca__n_components': Integer(1,len(digits.data[0])),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    }
}

tunedModel = {}

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=42)
for name, classDict in classifiers.items():
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('classify',classDict['model']())],
    verbose=False)

    tunedModel[name] = BayesSearchCV(
        pipe,
        classDict['paramSpace'],
        n_iter=30,
        cv=20,
        scoring='accuracy'
        # TODO: Use GridSearch for scoring criteria
        # NOTE: This will take 13 hours. Execute over night.
        # random_state=42 
        )
    tunedModel[name].fit(X_train, y_train)
    
    # search = BayesSearchCV(pipe, param_grid, n_jobs=2)
    # predicted = search.best_estimator_.predict(X_test)
    predicted = tunedModel[name].best_estimator_.predict(X_test)
    
    print(classification_report(y_test, predicted))
    sns.heatmap(confusion_matrix(y_test, predicted))
    plt.title(name + ' Confusion Matrix')
    plt.show()




In [ ]:
for name, model in tunedModel.items():
    _ = plot_objective(model.optimizer_results_[0])
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    plt.show()
    _ = plot_convergence(model.optimizer_results_[0])
    plt.show()

In [ ]:
'''Regressor data EDA'''
cal_housing = fetch_california_housing(as_frame=True)
# print(cal_housing.data)
# print(cal_housing.target)

outlierTable = PrettyTable(['Feature', 'Outlier Count'])

for column in cal_housing.data.columns:
    Q1 = cal_housing.data[column].quantile(0.25)
    Q3 = cal_housing.data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    outlierCount = np.array(cal_housing.data[column] >= upper).sum() + np.array(cal_housing.data[column] <= lower).sum()
    outlierTable.add_row([column, outlierCount])
    # print(f"{column}: {outlierCount}")

print(outlierTable)


In [ ]:
'''Data Cleaning and Pre-Processing'''
cleanData = cal_housing.data
cleanData['y'] = cal_housing.target

cleanData = cleanData.drop(columns=['Longitude', 'Latitude'])
for feature in ['AveBedrms', 'AveRooms', 'AveOccup', 'Population']:
    cleanData = cleanData[(np.abs(stats.zscore(cleanData[feature])) < 2.5)]
cleanTarget = cleanData['y'].to_list()

# DATA CLEANING TODOs
# TODO: Bin Lat/Long groupings into city/town clusters. look for available geo-fencing data for cluster labeling - can we do a graph of centroids on top of map?
# TODO: Fix Skew for Population, MedIncome, AvgOccup, AvgBedroom, Target
# TODO: Feature Engineering / Reduction
cleanData.drop(columns=['y'], inplace=True)

In [ ]:
'''Plot clean features to explore relationships'''
plotData = cleanData.copy()
plotData['y'] = cleanTarget.copy()

# sns.pairplot(plotData, hue='y')
sns.pairplot(plotData.drop(columns=['y']), kind='reg', plot_kws={'line_kws':{'color':'red'}})


In [ ]:
'''Show distribution of Target Value'''
sns.histplot(data=plotData, x='y')

In [ ]:
'''Plot data by Bin'''
bin1 = plotData[plotData['y'] < 1]
bin2 = plotData[plotData['y'] > 1][plotData['y'] <= 2]
bin3 = plotData[plotData['y'] > 2][plotData['y'] <= 3]
bin4 = plotData[plotData['y'] > 3][plotData['y'] <= 4]
bin5 = plotData[plotData['y'] > 4]

for index, bin in enumerate([bin1, bin2, bin3, bin4, bin5]):
    fig, axs = plt.subplots(ncols=2, nrows=2)
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    fig.suptitle('Bin'+str(index)+' - Pop '+str(len(bin)))
    sns.histplot(x='MedInc', data=bin, ax=axs[0,0])
    axs[0,0].set_ylim(bottom=0, top=800)
    axs[0,0].set_xlim(left=0, right=16)
    sns.regplot(x='MedInc', y='AveRooms', data=bin, ax=axs[0,1], line_kws={'color':'red'})
    axs[0,1].set_ylim(bottom=0, top=10)
    axs[0,1].set_xlim(left=0, right=16)
    sns.regplot(x='MedInc', y='AveBedrms', data=bin, ax=axs[1,0], line_kws={'color':'red'})
    axs[1,0].set_ylim(bottom=0, top=3.0)
    axs[1,0].set_xlim(left=0, right=16)
    # sns.countplot(x='MedInc', data=bin, ax=axs[1,1])
    sns.regplot(x='MedInc', y='AveOccup', data=bin, ax=axs[1,1], line_kws={'color':'red'})

    axs[1,1].set_ylim(bottom=0, top=25)
    axs[1,1].set_xlim(left=0, right=16)
    plt.show()


In [ ]:
'''Default Regressor Performance'''
modelData = cleanData.copy()

# TODO: How does normalization vs standardization impact model performance
transformPipeline = [
    ('scaler', StandardScaler()),
    # ('feature_reduction', PCA(n_components=5,iterated_power=7))
    ]

X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)
for regressor in [DecisionTreeRegressor, RandomForestRegressor]:
    pipe = Pipeline(transformPipeline + [('regress',regressor())], verbose=True)
    predicted = pipe.fit(X_train, y_train).predict(X_test)
    mse = mean_squared_error(y_test, predicted)
    print(regressor.__name__)
    print(mse)
    
    
    # TODO: Graph Regression Plane using skopt.plots
    # NOTE: PCA to force data into 3d space
    # feat1 = [x for x,_,_ in newX]
    # feat2 = [x for _,x,_ in newX]
    # feat3 = [x for _,_,x in newX]
    
    # fig = plt.figure()
    # ax = fig.add_subplot(projection='3d')
    
    # ax.scatter(feat1, feat2, feat3, label='Data')
    # ax.plot(y_test, predicted, color='red', label='Regression Line')
    # ax.set_xlabel('PCA 1')
    # ax.set_ylabel('PCA 2')
    # ax.set_zlabel('PCA 3')
    # ax.legend()
    # plt.title((f'Linear Regression (MSE: {mse:.2f})'))
    # plt.show()


In [ ]:
'''Tuned Regressor Performance'''
# TODO: Explore how increased demetionality in the parameter space impacts optimization performance
regressors = {
    DecisionTreeRegressor.__name__: {
        'model': DecisionTreeRegressor,
        'paramSpace': {
            'regress__criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'regress__splitter': ['best', 'random'],
            'regress__max_depth': Integer(2, 1000),
            'regress__min_samples_split': Real(0.01, 0.9),
            'regress__min_samples_leaf': Real(0.01, 0.9),
            'regress__min_weight_fraction_leaf': Real(0.0, 0.5),
            'regress__max_features': Real(0.01, 0.5),
            'regress__max_leaf_nodes': Integer(2, 1000),
            'regress__min_impurity_decrease': Real(0.0, 0.9),
            'regress__ccp_alpha': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(modelData.columns)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    },
    RandomForestRegressor.__name__: {
        'model': RandomForestRegressor,
        'paramSpace': {
            'regress__n_estimators': Integer(50, 500),
            'regress__criterion': Categorical(['squared_error', 'friedman_mse', 'absolute_error', 'poisson']),
            'regress__max_depth': Integer(2, 1000), 
            'regress__min_samples_split': Real(0.01, 0.9),
            'regress__min_samples_leaf': Real(0.01, 0.9),
            'regress__min_weight_fraction_leaf': Real(0.01, 0.5),
            'regress__max_features': Real(0.01,0.9),
            # 'regress__max_features': Categorical(['sqrt', 'log2']), 
            'regress__max_leaf_nodes': Integer(2,1000),
            'regress__min_impurity_decrease': Real(0.01, 0.9),
            # 'regress__bootstrap': [True, False],
            'regress__oob_score': [True, False],
            'regress__warm_start': [True, False],
            'regress__ccp_alpha': Real(0.01, 0.9),
            'regress__max_samples': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(modelData.columns)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    }
}

tunedModel = {}
X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)

for name, regDict in regressors.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('regress',regDict['model']())])
    tunedModel[name] = BayesSearchCV(
        pipe,
        regDict['paramSpace'],
        n_iter=30,
        cv=20,
        # scoring=scoreModel
        # scoring = scoringCriteria[i]
        # TODO: Use GridSearchCV for scoringCriteria param space
        # NOTE: This will take 13 hours. Execute over night.
        )
    tunedModel[name].fit(X_train, y_train)
    predicted = tunedModel[name].best_estimator_.predict(X_test)
    print('Evaluation Metric:', tunedModel[name].get_params()['scoring'])
    print("val. score: %s" % tunedModel[name].best_score_)
    print("test score: %s" % tunedModel[name].score(X_test, y_test))
    

In [ ]:
'''Explore Optimizer'''
# tunedModel[DecisionTreeRegressor.__name__].optimizer_results_
# .models[-1]
# print([x for x in regressors[DecisionTreeRegressor.__name__]['paramSpace'].keys()])

In [ ]:
'''Plot Optimization'''
for name, model in tunedModel.items():
    _ = plot_objective(result=model.optimizer_results_[0])
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    plt.show()
    _ = plot_convergence(model.optimizer_results_[0])
    plt.show()
